In [1]:
%load_ext autoreload
%autoreload 2
import json
import sys
import os
import pandas as pd
from math import floor
import numpy as np
import pickle
sys.path.append(os.path.abspath('../../../utils'))
from protocol_system_compiler import *

In [2]:
fluids = pd.read_csv('fluids.csv')

In [3]:
fluids

,NodeID,FluidType,BufferName,LoadVolume
0,ot2,source,ot2,3
1,2,source,wash buffer,40
2,3,source,imaging buffer,21
3,4,source,tcep,20
4,5,source,2x-SSC,50
5,6,source,universal_fluors,15
6,7,source,NaN,0
7,8,source,NaN,0
8,waste,destination,waste_tank,0


In [61]:
labware = {'tips_200':{
    'api_name': 'opentrons_96_filtertiprack_200ul',
    'location': '1',
    'creator': 'opentrons'
},
           'tips_1000':{
    'api_name': 'opentrons_96_filtertiprack_1000ul',
    'location': '3',
    'creator': 'opentrons'
},
              'reservoir1':{
    'api_name': 'nest_12_reservoir_15ml',
    'location': '5',
    'creator': 'opentrons'
                },
              'wellplate1':{
    'api_name': 'nest_96_wellplate_2ml_deep',
    'location': '7',
    'creator': 'opentrons'
                },
              'syringe_adapter':{
    'api_name': 'yeolab_1_bd_syringe_adap_3000ul',
    'location': '10',
    'creator': 'custom'
                },
              }
hardware = {'p200':{
    'api_name': 'p300_single_gen2',
    'mount': 'left',
    'tip_racks': ['tips_200']},
            'p1000':{
    'api_name': 'p1000_single_gen2',
    'mount': 'right',
    'tip_racks': ['tips_1000']}}

metadata = {'apiLevel': '2.14'}

ot2_config = {'labware':labware,
              'hardware':hardware,
              'metadata':metadata}

In [63]:
reagents = {}


In [64]:
def generate_well_pos(well_num,total=96,ncol=12,nrow=8):
    row = well_num//ncol
    col = well_num%ncol
    return chr(65+row)+str(col+1)

In [65]:
num_rounds = 2
for i in range(num_rounds):
    reagents['round'+str(i+1)] = {'labware':'wellplate1',
                                'location':generate_well_pos(i+2),
                                'volume':750,
                           'hardware':'p1000',
                           'repeat':2}

In [66]:
def get_fluid_node(buffer):
    return fluids[fluids['BufferName']==buffer]['NodeID'].iloc[0]

In [73]:
experiment = {}

In [74]:
step_n = 0

In [75]:
reagents['round2']

{'labware': 'wellplate1',
 'location': 'A4',
 'volume': 750,
 'hardware': 'p1000',
 'repeat': 2}

# Testing for imaging

In [76]:
for round_n in range(1,3):
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'liquid_handler'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['reagent'] = f'round{round_n}'
    experiment[step_n]['step_metadata']['info'] = reagents[f'round{round_n}']
    experiment[step_n]['step_metadata']['destination'] = 'syringe_adapter'
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('ot2')
    experiment[step_n]['step_metadata']['volume'] = 1.5
    experiment[step_n]['step_metadata']['speed'] = 1
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'wait'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['wait_time'] = 30 # 15 minutes
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'image'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['filename'] = f'round_{round_n}'
    experiment[step_n]['step_metadata']['callibrate_af'] = True
    experiment[step_n]['slack_notify'] = True
    step_n += 1

In [77]:
experiment

{0: {'step_type': 'liquid_handler',
  'step_metadata': {'reagent': 'round1',
   'info': {'labware': 'wellplate1',
    'location': 'A3',
    'volume': 750,
    'hardware': 'p1000',
    'repeat': 2},
   'destination': 'syringe_adapter'},
  'slack_notify': False},
 1: {'step_type': 'fluid',
  'step_metadata': {'fluid': 'ot2', 'volume': 1.5, 'speed': 1},
  'slack_notify': False},
 2: {'step_type': 'wait',
  'step_metadata': {'wait_time': 30},
  'slack_notify': False},
 3: {'step_type': 'image',
  'step_metadata': {'filename': 'round_1', 'callibrate_af': True},
  'slack_notify': True},
 4: {'step_type': 'liquid_handler',
  'step_metadata': {'reagent': 'round2',
   'info': {'labware': 'wellplate1',
    'location': 'A4',
    'volume': 750,
    'hardware': 'p1000',
    'repeat': 2},
   'destination': 'syringe_adapter'},
  'slack_notify': False},
 5: {'step_type': 'fluid',
  'step_metadata': {'fluid': 'ot2', 'volume': 1.5, 'speed': 1},
  'slack_notify': False},
 6: {'step_type': 'wait',
  'step

# Testing for OT2

In [18]:
for round_n in range(1,2):
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'liquid_handler'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['reagent'] = f'round{round_n}'
    experiment[step_n]['step_metadata']['info'] = reagents[f'round{round_n}']
    experiment[step_n]['step_metadata']['destination'] = 'syringe_adapter'
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'fluid'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['fluid'] = get_fluid_node('ot2')
    experiment[step_n]['step_metadata']['volume'] = 1.5
    experiment[step_n]['step_metadata']['speed'] = 1
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    """
    experiment[step_n] = {}
    experiment[step_n]['step_type'] = 'wait'
    experiment[step_n]['step_metadata'] = {}
    experiment[step_n]['step_metadata']['wait_time'] = 10 # 15 minutes
    experiment[step_n]['slack_notify'] = False
    step_n += 1
    """

In [33]:
experiment

{0: {'step_type': 'liquid_handler',
  'step_metadata': {'reagent': 'round1',
   'info': {'labware': 'wellplate1',
    'location': 'A2',
    'volume': 750,
    'hardware': 'p1000',
    'repeat': 2},
   'destination': 'yeolab_1_bd_syringe_adap_3000ul'},
  'slack_notify': False},
 1: {'step_type': 'fluid',
  'step_metadata': {'fluid': 'ot2', 'volume': 1.5, 'speed': 1},
  'slack_notify': False},
 2: {'step_type': 'wait',
  'step_metadata': {'wait_time': 900},
  'slack_notify': False},
 3: {'step_type': 'image',
  'step_metadata': {'filename': 'round_1', 'callibrate_af': True},
  'slack_notify': True},
 4: {'step_type': 'liquid_handler',
  'step_metadata': {'reagent': 'round2',
   'info': {'labware': 'wellplate1',
    'location': 'A3',
    'volume': 750,
    'hardware': 'p1000',
    'repeat': 2},
   'destination': 'yeolab_1_bd_syringe_adap_3000ul'},
  'slack_notify': False},
 5: {'step_type': 'fluid',
  'step_metadata': {'fluid': 'ot2', 'volume': 1.5, 'speed': 1},
  'slack_notify': False},


In [78]:
with open('experiment.json', 'w+') as f:
    json.dump(experiment, f, indent=4)


In [62]:
with open('ot2_config.json', 'w+') as f:
    json.dump(ot2_config, f, indent=4)


In [41]:
with open('experiment.json', 'r') as f:
    e = json.load(f)

In [44]:
e['3']['step_metadata']['callibrate_af']

True

In [16]:
def get_tip_num(experiment,step_num):
    # figure out which steps are liquid handling rounds
    liquid_handling_steps = []
    for step, value in experiment.items():
        if value['step_type'] == 'liquid_handler':
            liquid_handling_steps.append(step)
    tip_num = liquid_handling_steps.index(step_num)
    return tip_num

In [22]:
# keys needed in oni_params dictionary
# 'xy_start', 'start_z', 'offset_z', "oni_json", 'config', 'picked_fovs' (bool if fovs were picked), 'fov_positions'
# within oni_json
# 'microscope_name', 'cropping'
# within 'config'
# 'steps' <-- contains light_program, 'movementOptions' <-- ['customXYPositions_mm'['position'], 'numberOfStepsInXY','fovIncrement_mm','zStackStartPosition_um','numberOfZSlices','spacingBetweenZSlices_um','exposure_ms','power']
#|___list of steps each with keys 'exposure_ms', 'power'


# Creating ONI imaging parameters

In [18]:
old_oni_config = {"doSIMDeconvolution":False,"imagingMode":"NORMAL","lightProgramToUse":"None","movementOptions":{"autoFocusWhen":0,"customXYPositions_mm":[{"offset":0.0,"position":{"x":0.529711,"y":-0.11074500000000001,"z":-0.45599900000000004}}],"disableZLockDuringMove":False,"doMoveXY":True,"doMoveZ":True,"doZStack":True,"fovIncrement_mm":{"x":0.08345999908447266,"y":0.13337998962402345},"movementMode":0,"numberOfStepsInXY":{"x":40,"y":10},"numberOfZSlices":10,"spacingBetweenZSlices_um":0.444,"startPositionXYZ_mm":{"offset":0.0,"position":{"x":0.52971,"y":-0.11074500000000001,"z":-0.45599900000000004}},"zReferenceMode":1,"zStackMovementMode":0,"zStackStartPosition_um":2.0},"numFramesPerXYZPosition":1,"orderPriority":2,"savingOptions":{"combinationType":"cropped","combineDatasets":False,"dataSetDirectory":"im_test","dataSetTag":"config","disableTiffSaving":False,"doLocalisation":True,"exposure_ms":33.0,"savingMode":"DataSetPerPosition","simSaveSummedImage":False,"useAdvancedMacros":False},"steps":[{"exposure_ms":33.0,"imagingMode":"NORMAL","lightStates":[{"on":True,"powerControlMethod":0,"value":10.0},{"on":False,"powerControlMethod":0,"value":0.0},{"on":False,"powerControlMethod":0,"value":0.0},{"on":False,"powerControlMethod":0,"value":0.0}],"numberOfFrames":1,"transState":{"colour":2,"intensity":0.0,"on":False}}],"timelapseOptions":{"doTimelapse":False,"leaveAutoFocusOnBetweenRepetitions":False,"numberOfRepetitionsOfMultiAcquisition":1,"repetitionMode":"FixedNumberOfRepetions","repetitionPeriod_ms":0,"returnToFirstPositionAtEnd":True,"totalTime_ms":0},"version":"Development build: Apr  9 2023 22:54:56 Version: 1.19.7.20230409223555 - 28f00b5","waitTimeAfterStageMovement_ms":0,"waitTimeAfterSwitchingLasersOn_ms":50}

In [19]:
import pprint

In [20]:
pprint.pprint(old_oni_config)

{'doSIMDeconvolution': False,
 'imagingMode': 'NORMAL',
 'lightProgramToUse': 'None',
 'movementOptions': {'autoFocusWhen': 0,
                     'customXYPositions_mm': [{'offset': 0.0,
                                               'position': {'x': 0.529711,
                                                            'y': -0.11074500000000001,
                                                            'z': -0.45599900000000004}}],
                     'disableZLockDuringMove': False,
                     'doMoveXY': True,
                     'doMoveZ': True,
                     'doZStack': True,
                     'fovIncrement_mm': {'x': 0.08345999908447266,
                                         'y': 0.13337998962402345},
                     'movementMode': 0,
                     'numberOfStepsInXY': {'x': 40, 'y': 10},
                     'numberOfZSlices': 10,
                     'spacingBetweenZSlices_um': 0.444,
                     'startPositionXYZ_mm': {'offset

In [21]:
{
    "xy_start": 0,
    "start_z": -466,
    "offset_z": -466
}

{'xy_start': 0, 'start_z': -466, 'offset_z': -466}

In [79]:
oni_params = {
    "cropping": {
        "top-left": [130, 5],
        "top-right": [140, 595],
        "width": 428,
        "height": 684
    },
    "safe_focus": {
        "upper_thresh": 2000,
        "lower_thresh": -450
    },
    "microscope_name": "60xAALZ7W2T",
    "xy_positions_mm": {},
    "z_relative_positions_um": [],
    "light_program":[{'laser_power': [10.0, 0.0, 0.0, 0.0], 'exposure_ms': 33}],
    "start_z": -675,
    "num_z_slices": 10,
    "relative_top_z_um": 2.0,
    "relative_bottom_z_um": -2.0,
    'fovIncrement_mm': {'x': 0.08345999908447266,'y': 0.13337998962402345} # ONI default with no overlap
}

In [80]:
def _make_tiles(starting_pos,n_x,n_y,x_inc_um,y_inc_um):
        x_start = starting_pos['x']
        y_start = starting_pos['y']
        positions = []
        xs = list(range(0,n_x))
        ys = list(range(0,n_y))
        if (len(ys) % 2) == 0:
            all_xs = (xs + list(reversed(xs)))*floor(len(ys)/2)
        else:
            all_xs = (xs + list(reversed(xs)))*floor(len(ys)/2)
            all_xs += xs
        all_ys = []
        for y in ys:
            all_ys += [y]*len(xs)
        tmp_positions = list(zip(all_xs,all_ys))
        positions = []
        for pos in tmp_positions:
            positions.append([x_start + (pos[0] * x_inc_um), y_start + (pos[1] * y_inc_um)])
        return positions

# custom_positions = None
def position_generator(oni_params,custom_positions,starting_xy_um={'x':0,'y':0},tile_num_x=1,tile_num_y=1,overlap=0.0):
    if custom_positions is not None:
        oni_params['xy_positions_mm'] = custom_positions
    else:
        x_inc_um = (oni_params['fovIncrement_mm']['x'] * 1000) * (1 - overlap)
        y_inc_um = (oni_params['fovIncrement_mm']['y'] * 1000) * (1 - overlap)
        oni_params['xy_positions_mm'] = _make_tiles(starting_xy_um,tile_num_x,tile_num_y,x_inc_um,y_inc_um)
    # generate relative_zs
    oni_params['z_relative_positions_um'] = np.linspace(oni_params['relative_top_z_um'],oni_params['relative_bottom_z_um'],oni_params['num_z_slices']).tolist()
    return oni_params

In [81]:
oni_params = position_generator(oni_params,None,{'x':5.25*1000,'y':2*1000},10,5,0.0)

In [82]:
oni_params

{'cropping': {'top-left': [130, 5],
  'top-right': [140, 595],
  'width': 428,
  'height': 684},
 'safe_focus': {'upper_thresh': 2000, 'lower_thresh': -450},
 'microscope_name': '60xAALZ7W2T',
 'xy_positions_mm': [[5250.0, 2000.0],
  [5333.459999084473, 2000.0],
  [5416.919998168945, 2000.0],
  [5500.379997253418, 2000.0],
  [5583.839996337891, 2000.0],
  [5667.299995422363, 2000.0],
  [5750.759994506836, 2000.0],
  [5834.219993591309, 2000.0],
  [5917.679992675781, 2000.0],
  [6001.139991760254, 2000.0],
  [6001.139991760254, 2133.3799896240234],
  [5917.679992675781, 2133.3799896240234],
  [5834.219993591309, 2133.3799896240234],
  [5750.759994506836, 2133.3799896240234],
  [5667.299995422363, 2133.3799896240234],
  [5583.839996337891, 2133.3799896240234],
  [5500.379997253418, 2133.3799896240234],
  [5416.919998168945, 2133.3799896240234],
  [5333.459999084473, 2133.3799896240234],
  [5250.0, 2133.3799896240234],
  [5250.0, 2266.759979248047],
  [5333.459999084473, 2266.759979248047

In [83]:
with open('oni_params.pkl', 'wb') as f:
    pickle.dump(oni_params, f)

In [26]:
parent_dir = os.path.abspath('../../../')

In [27]:
compile_protocol('Pepper-ONI_v1-001','test',parent_dir)

Checking for:
/Users/noorsherahmed/Library/CloudStorage/Dropbox/Yeo_lab/Noor/gabeen/system-files/Pepper-ONI_v1-001/comports.json
/Users/noorsherahmed/Library/CloudStorage/Dropbox/Yeo_lab/Noor/gabeen/system-files/Pepper-ONI_v1-001/fluid_nodes.csv
/Users/noorsherahmed/Library/CloudStorage/Dropbox/Yeo_lab/Noor/gabeen/protocols/Pepper-ONI_v1-001/test/experiment.json
/Users/noorsherahmed/Library/CloudStorage/Dropbox/Yeo_lab/Noor/gabeen/system-files/Pepper-ONI_v1-001/fluid_edges.csv
/Users/noorsherahmed/Library/CloudStorage/Dropbox/Yeo_lab/Noor/gabeen/protocols/Pepper-ONI_v1-001/test/fluids.csv
/Users/noorsherahmed/Library/CloudStorage/Dropbox/Yeo_lab/Noor/gabeen/system-files/Pepper-ONI_v1-001/oni_params.json
all checks passed!
